In [1]:
from huawei import router1, router2, HuaweiRouter
from pathlib import Path

log_dir = Path("log")
config_router1 = Path("router1.txt")
config_router2 = Path("router2.txt")

In [4]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.configure(config_router1)
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.configure(config_router2)

log\Router1_20251124-142932.log
[INFO] Connected to Router1 (172.16.2.203)
[INFO] Applying configuration from router1.txt to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]dhcp enable
[Router1]ip pool net1
Info: The parameter is case insensitive. All uppercase letters in the string will be converted to lowercase letters.
Info: It is successful to create an IP address pool.
[Router1-ip-pool-net1]network 192.168.1.0 mask 255.255.255.0
[Router1-ip-pool-net1]gateway-list 192.168.1.1
[Router1-ip-pool-net1]quit
[Router1]observe-port interface GigabitEthernet 0/0/1
[Router1]interface GigabitEthernet0/0/0
[Router1-GigabitEthernet0/0/0]ip address 192.168.1.1 24
[Router1-GigabitEthernet0/0/0]dhcp select global
[Router1-GigabitEthernet0/0/0]mirror to observe-port both
[Router1-GigabitEthernet0/0/0]quit
[Router1]interface GigabitEthernet0/0/2
[Router1-GigabitEthernet0/0/2]ip address 192.168.2.1 24
[Router1-GigabitEthernet0/0/2]mirror to observe-port bo

In [5]:
ospf_commands = [
    "display ospf lsdb router",
    "display ospf lsdb network",
    "display ospf lsdb",
    "display ospf peer"
]
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands(ospf_commands)
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands(ospf_commands)

log\Router1_20251124-143535.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]display ospf lsdb router

	 OSPF Process 1 with Router ID 1.1.1.1
		         Area: 0.0.0.0
		 Link State Database 


  Type      : Router
  Ls id     : 2.2.2.2
  Adv rtr   : 2.2.2.2  
  Ls age    : 304 
  Len       : 48 
  Options   :  E  
  seq#      : 80000006 
  chksum    : 0x8a5b
  Link count: 2
   * Link ID: 192.168.2.2  
     Data   : 192.168.2.2  
     Link Type: TransNet     
     Metric : 1
   * Link ID: 192.168.3.0  
     Data   : 255.255.255.0 
     Link Type: StubNet      
     Metric : 1 
     Priority : Low

  Type      : Router
  Ls id     : 1.1.1.1
  Adv rtr   : 1.1.1.1  
  Ls age    : 305 
  Len       : 48 
  Options   :  E  
  seq#      : 80000006 
  chksum    : 0xcb25
  Link count: 2
   * Link ID: 192.168.1.0  
     Data   : 255.255.255.0 
     Link Type: StubNet      
     Metric : 1 
     Priority : Low
   * Link ID: 192.1

In [6]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands("display ip routing-table")
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands("display ip routing-table")

log\Router1_20251124-144941.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]display ip routing-table
Route Flags: R - relay, D - download to fib, T - to vpn-instance
------------------------------------------------------------------------------
Routing Tables: Public
         Destinations : 14       Routes : 14       

Destination/Mask    Proto   Pre  Cost      Flags NextHop         Interface

      127.0.0.0/8   Direct  0    0           D   127.0.0.1       InLoopBack0
      127.0.0.1/32  Direct  0    0           D   127.0.0.1       InLoopBack0
127.255.255.255/32  Direct  0    0           D   127.0.0.1       InLoopBack0
     172.16.0.0/16  Direct  0    0           D   172.16.2.203    GigabitEthernet0/0/8
   172.16.2.203/32  Direct  0    0           D   127.0.0.1       GigabitEthernet0/0/8
 172.16.255.255/32  Direct  0    0           D   127.0.0.1       GigabitEthernet0/0/8
    192.168.1.0/24  Direct  0    0           

In [11]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands("display ospf peer")

log\Router1_20251124-151150.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]display ospf peer

	 OSPF Process 1 with Router ID 3.3.3.3
		 Neighbors 

 Area 0.0.0.0 interface 192.168.2.1(GigabitEthernet0/0/2)'s neighbors
 Router ID: 2.2.2.2          Address: 192.168.2.2     
   State: Full  Mode:Nbr is  Slave  Priority: 1
   DR: 192.168.2.1  BDR: 192.168.2.2  MTU: 0    
   Dead timer due in 38  sec 
   Retrans timer interval: 5 
   Neighbor is up for 00:00:21     
   Authentication Sequence: [ 0 ] 

[Router1]return
<Router1>
[INFO] Disconnected from Router1 (172.16.2.203)


In [18]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands(["ip pool net4", "network 192.168.4.0 mask 255.255.255.0", "gateway-list 192.168.4.1", "quit", "interface GigabitEthernet0/0/3", "ip address 192.168.4.1 24", "dhcp select global", "undo shutdown", "quit"])
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands(["ip pool net5", "network 192.168.5.0 mask 255.255.255.0", "gateway-list 192.168.5.1", "quit", "interface GigabitEthernet0/0/7", "ip address 192.168.5.1 24", "dhcp select global", "undo shutdown", "quit"])

log\Router1_20251124-153219.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]ip pool net4
Info: The parameter is case insensitive. All uppercase letters in the string will be converted to lowercase letters.
Info: It is successful to create an IP address pool.
[Router1-ip-pool-net4]network 192.168.4.0 mask 255.255.255.0
[Router1-ip-pool-net4]gateway-list 192.168.4.1
[Router1-ip-pool-net4]quit
[Router1]interface GigabitEthernet0/0/3
[Router1-GigabitEthernet0/0/3]ip address 192.168.4.1 24
[Router1-GigabitEthernet0/0/3]dhcp select global
[Router1-GigabitEthernet0/0/3]undo shutdown
Info: Interface GigabitEthernet0/0/3 is not shutdown.
[Router1-GigabitEthernet0/0/3]quit
[Router1]return
<Router1>
[INFO] Disconnected from Router1 (172.16.2.203)
log\Router2_20251124-153233.log
[INFO] Connected to Router2 (172.16.2.204)
system-view
Enter system view, return user view with Ctrl+Z.
[Router2]ip pool net5
Info: It is successful to c

In [19]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands(["ospf", "area 0", "network 192.168.4.0 0.0.0.255"])
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands(["ospf", "area 0", "network 192.168.5.0 0.0.0.255"])

log\Router1_20251124-154031.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]ospf
[Router1-ospf-1]area 0
[Router1-ospf-1-area-0.0.0.0]network 192.168.4.0 0.0.0.255
[Router1-ospf-1-area-0.0.0.0]return
<Router1>
[INFO] Disconnected from Router1 (172.16.2.203)
log\Router2_20251124-154045.log
[INFO] Connected to Router2 (172.16.2.204)
system-view
Enter system view, return user view with Ctrl+Z.
[Router2]ospf
[Router2-ospf-1]area 0
[Router2-ospf-1-area-0.0.0.0]network 192.168.5.0 0.0.0.255
[Router2-ospf-1-area-0.0.0.0]return
<Router2>
[INFO] Disconnected from Router2 (172.16.2.204)


In [20]:
ospf_commands = [
    "display ospf lsdb router",
    "display ospf lsdb network",
    "display ospf lsdb",
    "display ospf peer"
]
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands(ospf_commands)
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands(ospf_commands)

log\Router1_20251124-154436.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]display ospf lsdb router

	 OSPF Process 1 with Router ID 3.3.3.3
		         Area: 0.0.0.0
		 Link State Database 


  Type      : Router
  Ls id     : 2.2.2.2
  Adv rtr   : 2.2.2.2  
  Ls age    : 189 
  Len       : 60 
  Options   :  E  
  seq#      : 8000001c 
  chksum    : 0x242d
  Link count: 3
   * Link ID: 192.168.2.1  
     Data   : 192.168.2.2  
     Link Type: TransNet     
     Metric : 1
   * Link ID: 192.168.3.0  
     Data   : 255.255.255.0 
     Link Type: StubNet      
     Metric : 1 
     Priority : Low
   * Link ID: 192.168.5.0  
     Data   : 255.255.255.0 
     Link Type: StubNet      
     Metric : 1 
     Priority : Low

  Type      : Router
  Ls id     : 1.1.1.1
  Adv rtr   : 1.1.1.1  
  Ls age    : 2849 
  Len       : 48 
  Options   :  E  
  seq#      : 8000000a 
  chksum    : 0xc329
  Link count: 2
   * Link ID: 192.

In [21]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands("display interface brief")
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands("display interface brief")

log\Router1_20251124-154525.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]display interface brief
PHY: Physical
*down: administratively down
(l): loopback
(s): spoofing
(b): BFD down
^down: standby
(e): ETHOAM down
(v): VirtualPort
InUti/OutUti: input utility/output utility
Interface                   PHY   Protocol  InUti OutUti   inErrors  outErrors
GigabitEthernet0/0/0        up    up        0.01%  0.01%          0          0
GigabitEthernet0/0/1        down  down         0%     0%          0          0
GigabitEthernet0/0/2        up    up        0.01%  0.01%          0          0
GigabitEthernet0/0/3        up    up           0%  0.01%          0          0
GigabitEthernet0/0/4        down  down         0%     0%          0          0
GigabitEthernet0/0/5        down  down         0%     0%          0          0
GigabitEthernet0/0/6        down  down         0%     0%          0          0
GigabitEthernet0/0/7  

In [ ]:
with HuaweiRouter(router1, log_dir=log_dir) as R1:
    R1.send_commands("display interface")
with HuaweiRouter(router2, log_dir=log_dir) as R2:
    R2.send_commands("display interface")

log\Router1_20251124-154600.log
[INFO] Connected to Router1 (172.16.2.203)
system-view
Enter system view, return user view with Ctrl+Z.
[Router1]display interface
GigabitEthernet0/0/0 current state : UP
Line protocol current state : UP
Last line protocol up time : 2025-11-24 06:31:31
Description:HUAWEI, AR Series, GigabitEthernet0/0/0 Interface
Route Port,The Maximum Transmit Unit is 1500
Internet Address is 192.168.1.1/24
IP Sending Frames' Format is PKTFMT_ETHNT_2, Hardware address is 64bf-6ba7-a11d
Last physical up time   : 2025-11-24 06:22:42
Last physical down time : 2025-11-24 06:13:09
Current system time: 2025-11-24 07:47:59
Port Mode: COMMON COPPER
Speed : 1000,  Loopback: NONE
Duplex: FULL,  Negotiation: ENABLE
Mdi   : AUTO,  Clock   : -
Last 300 seconds input rate 1496 bits/sec, 0 packets/sec
Last 300 seconds output rate 112 bits/sec, 0 packets/sec
Input peak rate 8136 bits/sec,Record time: 2025-11-24 06:35:18
Output peak rate 1656 bits/sec,Record time: 2025-11-24 06:35:18

I